In [ ]:
from os.path import join

import boto3
from botocore.exceptions import ClientError

from rastervision.common.settings import results_path

best_densenet_run_names = {
    'tagging/7_5_17/ensemble/0': 1,
    'tagging/7_5_17/ensemble/1': 1,
    'tagging/7_5_17/ensemble/2': 1,
    'tagging/7_5_17/ensemble/3': 1,
    'tagging/7_5_17/ensemble/4': 1,
    'tagging/7_11_17/dense_ensemble2/0': 1,
    'tagging/7_11_17/dense_ensemble2/1': 1,
    'tagging/7_11_17/dense_ensemble2/2': 1,
    'tagging/7_11_17/dense_ensemble2/3': 1,
    'tagging/7_11_17/dense_ensemble2/4': 1,
    'tagging/7_7_17/jpg_ensemble_0/0': 1,
    'tagging/7_7_17/jpg_ensemble_0/1': 1,
    'tagging/7_7_17/jpg_ensemble_0/2': 1,
    'tagging/7_7_17/jpg_ensemble_0/3': 1
}

other_densenet_run_names = {
    'tagging/6_30_17/dense121_dropout_1': 1,
    'tagging/6_30_17/dense121_dropout_2': 1,
    'tagging/6_30_17/dense121_dropout_3': 1,
    'tagging/6_29_17/dense121_3x10epochs_0': 1
}

resnet_run_names = {
    'tagging/7_7_17/baseline_cyclic_1': 1,
    'tagging/7_7_17/baseline_cyclic_1200steps': 1,
    'tagging/7_7_17/baseline_cyclic_2': 1, # bad
    'tagging/7_6_17/baseline_cyclic': 1, # v bad
    'tagging/6_28_17/baseline_testrot_0': 1,
    'tagging/7_10_17/cyclic_2400steps': 1
}

tiff_run_names = {
    'tagging/7_3_17/baseline-branch-tiffdrop': 1,
    'tagging/7_3_17/dual_resnet-stage4-moretrain': 1,
    'tagging/7_3_17/dual_resnet-stage5-moretrain': 1,
    'tagging/7_2_17/dual_resnet_30epoch': 1,
    'tagging/7_7_17/tif_ensemble_0/0': 1,
    'tagging/7_7_17/tif_ensemble_0/1': 1,
    'tagging/7_7_17/tif_ensemble_0/2': 1,
    'tagging/7_7_17/tif_ensemble_0/3': 1,
    'tagging/7_2_17/IRRGtiff_v5actual_dense121_3x10epochs_0': 1,
    'tagging/6_29_17/IRRGBtiff_v5_dense121_3x10epochs_0': 1
}

all_run_names = {**best_densenet_run_names, **other_densenet_run_names, **resnet_run_names, **tiff_run_names}

test_pred_fn = 'test_preds.csv'
scores_fn = 'scores.json'
s3_bucket = 'raster-vision'


def s3_download(run_name, file_name, new_file_name=None):
    if new_file_name is None:
        new_file_name = file_name
    s3_key = 'results/{}/{}'.format(run_name, file_name)
    run_path = join('/opt/data/results/', run_name, new_file_name)
    s3 = boto3.resource('s3')
    s3.Bucket(s3_bucket).download_file(s3_key, run_path)


for run_name in all_run_names:
    try:
        s3_download(run_name, test_pred_fn)
    except:
        s3_download(run_name, 'test_predictions.csv', 'test_preds.csv')
        
        
    s3_download(run_name, scores_fn)

In [16]:
import json

run_f2_list = []
for run_name in all_run_names:
    run_path = join(results_path, run_name)
    scores_path = join(run_path, 'scores.json')
    with open(scores_path, 'r') as scores_file:
        scores = json.load(scores_file)
        if 'f2' in scores:
            f2 = scores['f2']
        elif 'f2_samples' in scores:
            f2 = scores['f2_samples']
    run_f2_list.append((run_name, f2))

run_f2_list = sorted(run_f2_list, key=lambda x: x[1])
print(run_f2_list)
print(len(run_f2_list))


[('tagging/6_30_17/dense121_dropout_3', 0.89206), ('tagging/7_6_17/baseline_cyclic', 0.90561), ('tagging/6_30_17/dense121_dropout_2', 0.91174), ('tagging/7_7_17/tif_ensemble_0/1', 0.91378), ('tagging/7_7_17/tif_ensemble_0/0', 0.9153), ('tagging/7_7_17/tif_ensemble_0/2', 0.91531), ('tagging/7_7_17/tif_ensemble_0/3', 0.9159), ('tagging/7_10_17/cyclic_2400steps', 0.92227), ('tagging/7_7_17/baseline_cyclic_2', 0.92239), ('tagging/7_2_17/dual_resnet_30epoch', 0.92289), ('tagging/6_30_17/dense121_dropout_1', 0.92355), ('tagging/7_3_17/baseline-branch-tiffdrop', 0.92515), ('tagging/6_28_17/baseline_testrot_0', 0.9253781318970175), ('tagging/7_3_17/dual_resnet-stage4-moretrain', 0.92547), ('tagging/7_7_17/baseline_cyclic_1200steps', 0.92613), ('tagging/7_3_17/dual_resnet-stage5-moretrain', 0.92624), ('tagging/7_2_17/IRRGtiff_v5actual_dense121_3x10epochs_0', 0.92637), ('tagging/6_29_17/dense121_3x10epochs_0', 0.9278674897960667), ('tagging/7_7_17/baseline_cyclic_1', 0.92799), ('tagging/7_7_17/j

In [22]:
decent_f2_list = list(filter(lambda x: x[1] > 0.92, run_f2_list))
print(decent_f2_list)
print(len(decent_f2_list))
decent_run_names = list(map(lambda x: x[0], decent_f2_list))
print(decent_run_names)

[('tagging/7_10_17/cyclic_2400steps', 0.92227), ('tagging/7_7_17/baseline_cyclic_2', 0.92239), ('tagging/7_2_17/dual_resnet_30epoch', 0.92289), ('tagging/6_30_17/dense121_dropout_1', 0.92355), ('tagging/7_3_17/baseline-branch-tiffdrop', 0.92515), ('tagging/6_28_17/baseline_testrot_0', 0.9253781318970175), ('tagging/7_3_17/dual_resnet-stage4-moretrain', 0.92547), ('tagging/7_7_17/baseline_cyclic_1200steps', 0.92613), ('tagging/7_3_17/dual_resnet-stage5-moretrain', 0.92624), ('tagging/7_2_17/IRRGtiff_v5actual_dense121_3x10epochs_0', 0.92637), ('tagging/6_29_17/dense121_3x10epochs_0', 0.9278674897960667), ('tagging/7_7_17/baseline_cyclic_1', 0.92799), ('tagging/7_7_17/jpg_ensemble_0/3', 0.9282), ('tagging/7_7_17/jpg_ensemble_0/0', 0.9285), ('tagging/7_11_17/dense_ensemble2/0', 0.92871), ('tagging/7_7_17/jpg_ensemble_0/1', 0.929), ('tagging/7_5_17/ensemble/0', 0.9295), ('tagging/7_11_17/dense_ensemble2/4', 0.92978), ('tagging/7_5_17/ensemble/1', 0.92978), ('tagging/7_5_17/ensemble/4', 0.92

In [24]:
import numpy as np
import math

from rastervision.tagging.data.planet_kaggle import TagStore

run_names = decent_run_names
run_name = run_names[0]
run_path = join(results_path, run_name)
test_pred_path = join(run_path, test_pred_fn)
test_file_inds = list(TagStore(test_pred_path).file_ind_to_tags.keys())

test_preds_list = []
for run_name in run_names:
    run_path = join(results_path, run_name)
    test_pred_path = join(run_path, test_pred_fn) 
    test_preds = TagStore(test_pred_path).get_tag_array(test_file_inds)
    test_preds_list.append(np.expand_dims(test_preds, axis=2))

test_preds = np.concatenate(test_preds_list, axis=2)
test_preds_sum = np.sum(test_preds, axis=2)
weight_sum = test_preds.shape[2]
true_thresh = math.ceil(weight_sum / 2)
test_preds = test_preds_sum >= true_thresh    

FileNotFoundError: [Errno 2] No such file or directory: '/opt/data/results/tagging/6_29_17/dense121_3x10epochs_0/test_preds.csv'

In [11]:
out_path = '/opt/data/jupyter/out/test_preds.csv'
tag_store = TagStore()
for i in range(test_preds.shape[0]):
    tag_store.add_tags(test_file_inds[i], test_preds[i, :])

tag_store.save(out_path)